In [ ]:
import pandas as pd

In [ ]:
# Load both CSV files
movies = pd.read_csv("/content/tmdb_5000_movies.csv")
credits = pd.read_csv("/content/tmdb_5000_credits.csv")

In [ ]:
# Merge datasets on 'title'
movies = movies.merge(credits, on='title')

# Check shape and preview
print(movies.shape)
movies.head()

(4809, 23)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [ ]:
# Rename columns to standard names
movies = movies.rename(columns={
    'movie_id_y': 'movie_id',
    'cast_y': 'cast',
    'crew_y': 'crew'
})

# Now drop the duplicates we don’t need
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]


In [ ]:
print(movies.columns)

Index(['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew'], dtype='object')


In [ ]:
import ast


In [ ]:
import ast

def convert(obj):
    L = []
    for i in ast.literal_eval(obj):  # safely convert string to list of dicts
        L.append(i['name'])          # take only the 'name' value from each dictionary
    return L

movies['genres'] = movies['genres'].apply(convert)


In [ ]:
def convert_cast(obj):
    L = []
    count = 0
    for i in ast.literal_eval(obj):
        if count < 3:              # only take top 3 actors
            L.append(i['name'])
            count += 1
        else:
            break
    return L

movies['cast'] = movies['cast'].apply(convert_cast)


In [ ]:
def fetch_director(obj):
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            return [i['name']]     # return only director's name in a list
    return []

movies['crew'] = movies['crew'].apply(fetch_director)


In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)


In [ ]:
movies['overview'] = movies['overview'].fillna("")


In [ ]:
movies['overview'] = movies['overview'].apply(lambda x: x.split())


In [ ]:
movies['cast'] = movies['cast'].apply(lambda x: [i.replace(" ", "") for i in x])


In [ ]:
movies['overview'] = movies['overview'].fillna("")
movies['genres'] = movies['genres'].fillna("[]")
movies['keywords'] = movies['keywords'].fillna("[]")
movies['cast'] = movies['cast'].fillna("[]")
movies['crew'] = movies['crew'].fillna("[]")


In [ ]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']
movies['tags'] = movies['tags'].apply(lambda x: " ".join(x))


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
cv = CountVectorizer(max_features=5000, stop_words='english')


In [ ]:
vectors = cv.fit_transform(movies['tags']).toarray()


In [ ]:
similarity = cosine_similarity(vectors)


In [ ]:
import requests

def fetch_poster(movie_title):
    api_key = "07f2e3aaaac8b022df203ebe794f9025"
    url = f"https://api.themoviedb.org/3/search/movie?api_key={api_key}&query={movie_title}"
    response = requests.get(url)
    data = response.json()

    if data['results']:
        poster_path = data['results'][0].get('poster_path')
        if poster_path:
            return f"https://image.tmdb.org/t/p/w500{poster_path}"

    return "https://via.placeholder.com/300x450.png?text=No+Image+Found"


In [ ]:
def recommend(movie):
    movie = movie.lower()
    if movie not in movies['title'].str.lower().values:
        return f"'{movie}' not found in database.", [], []

    idx = movies[movies['title'].str.lower() == movie].index[0]
    distances = similarity[idx]
    movie_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]

    recommended_titles = []
    recommended_posters = []

    for i in movie_list:
        title = movies.iloc[i[0]].title
        recommended_titles.append(title)
        recommended_posters.append(fetch_poster(title))

    return f"Movies similar to '{movies.iloc[idx].title}':", recommended_titles, recommended_posters


In [ ]:
message, titles, posters = recommend("Avatar")
for t, p in zip(titles, posters):
    print(t, p)


Aliens https://image.tmdb.org/t/p/w500/r1x5JGpyqZU8PYhbs4UcrO1Xb6x.jpg
Moonraker https://image.tmdb.org/t/p/w500/6LrJdXNmu5uHOVALZxVYd44Lva0.jpg
Alien https://image.tmdb.org/t/p/w500/vfrQk5IPloGg1v9Rzbh2Eg3VGyM.jpg
Silent Running https://image.tmdb.org/t/p/w500/ahasZ5HILZHtlrl6cbXgpfk0y8H.jpg
Spaceballs https://image.tmdb.org/t/p/w500/o624HTt93iIJIc1Sg5hNkDTnk5l.jpg


In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 104.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.9 MB/s eta 0:00:00


In [ ]:
import streamlit as st
import pandas as pd
import ast
import requests
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load datasets
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")
movies = movies.merge(credits, on='title')

movies = movies.rename(columns={
    'movie_id_y': 'movie_id',
    'cast_y': 'cast',
    'crew_y': 'crew'
})
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

def convert_cast(obj):
    L = []
    count = 0
    for i in ast.literal_eval(obj):
        if count < 3:
            L.append(i['name'])
            count += 1
        else:
            break
    return L

def fetch_director(obj):
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            return [i['name']]
    return []

movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)
movies['cast'] = movies['cast'].apply(convert_cast)
movies['crew'] = movies['crew'].apply(fetch_director)
movies['overview'] = movies['overview'].fillna("").apply(lambda x: x.split())
movies['cast'] = movies['cast'].apply(lambda x: [i.replace(" ", "") for i in x])
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']
movies['tags'] = movies['tags'].apply(lambda x: " ".join(x))

cv = CountVectorizer(max_features=5000, stop_words='english')
vectors = cv.fit_transform(movies['tags']).toarray()
similarity = cosine_similarity(vectors)

# Fetch poster from TMDb
def fetch_poster(movie_title):
    api_key = "07f2e3aaaac8b022df203ebe794f9025"
    url = f"https://api.themoviedb.org/3/search/movie?api_key={api_key}&query={movie_title}"
    response = requests.get(url)
    data = response.json()

    if data['results']:
        poster_path = data['results'][0].get('poster_path')
        if poster_path:
            return f"https://image.tmdb.org/t/p/w500{poster_path}"
    return "https://via.placeholder.com/300x450.png?text=No+Image+Found"

# Recommendation
def recommend(movie):
    movie = movie.lower()
    if movie not in movies['title'].str.lower().values:
        return [], []

    idx = movies[movies['title'].str.lower() == movie].index[0]
    distances = similarity[idx]
    movie_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]

    recommended_titles = []
    recommended_posters = []

    for i in movie_list:
        title = movies.iloc[i[0]].title
        recommended_titles.append(title)
        recommended_posters.append(fetch_poster(title))

    return recommended_titles, recommended_posters

# Streamlit UI
st.title('🎬 Movie Recommendation System (Content-Based)')
movie_name = st.text_input("Enter a movie name:")

if st.button("Recommend"):
    titles, posters = recommend(movie_name)

    if titles:
        for i in range(len(titles)):
            col1, col2 = st.columns([1, 2])
            with col1:
                st.image(posters[i], width=150)
            with col2:
                st.subheader(titles[i])
    else:
        st.error("Movie not found. Please check the spelling.")


2025-07-22 05:19:58.223 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-22 05:19:58.388 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-07-22 05:19:58.389 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-22 05:19:58.391 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-22 05:19:58.393 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-22 05:19:58.394 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-22 05:19:58.395 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-22 05:19:58.396 Thread 'MainThread': mi

In [ ]:
!pip install streamlit pyngrok


In [ ]:
app_code = """
import streamlit as st
import pandas as pd
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import requests

# Load data
movies = pd.read_csv('/content/tmdb_5000_movies.csv')
credits = pd.read_csv('/content/tmdb_5000_credits.csv')
movies = movies.merge(credits, on='title')
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

def convert_cast(obj):
    L = []
    count = 0
    for i in ast.literal_eval(obj):
        if count < 3:
            L.append(i['name'])
            count += 1
        else:
            break
    return L

def fetch_director(obj):
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            return [i['name']]
    return []

movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)
movies['cast'] = movies['cast'].apply(convert_cast)
movies['crew'] = movies['crew'].apply(fetch_director)

movies['overview'] = movies['overview'].fillna("")
movies['overview'] = movies['overview'].apply(lambda x: x.split())
movies['cast'] = movies['cast'].apply(lambda x: [i.replace(" ", "") for i in x])
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']
movies['tags'] = movies['tags'].apply(lambda x: " ".join(x))

cv = CountVectorizer(max_features=5000, stop_words='english')
vectors = cv.fit_transform(movies['tags']).toarray()
similarity = cosine_similarity(vectors)

# Your TMDB API key
API_KEY = '07f2e3aaaac8b022df203ebe794f9025'

def fetch_poster(title):
    idx = movies[movies['title'] == title].index[0]
    movie_id = movies.iloc[idx].movie_id
    response = requests.get(f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={API_KEY}")
    data = response.json()
    if 'poster_path' in data and data['poster_path']:
        return "https://image.tmdb.org/t/p/w500/" + data['poster_path']
    else:
        return "https://via.placeholder.com/500x750?text=No+Image"

def recommend(movie):
    movie = movie.lower()
    if movie not in movies['title'].str.lower().values:
        return [], []

    idx = movies[movies['title'].str.lower() == movie].index[0]
    distances = similarity[idx]
    movie_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    recommended_movies = []
    posters = []
    for i in movie_list:
        title = movies.iloc[i[0]].title
        recommended_movies.append(title)
        posters.append(fetch_poster(title))
    return recommended_movies, posters

# Streamlit UI
st.title("🎬 Movie Recommendation System")
movie_name = st.text_input("Enter a movie name:")

if st.button("Recommend"):
    titles, posters = recommend(movie_name)
    if titles:
        cols = st.columns(5)
        for i in range(5):
            with cols[i]:
                st.image(posters[i])
                st.caption(titles[i])
    else:
        st.warning("Movie not found. Please check the name.")
"""
with open("app.py", "w") as f:
    f.write(app_code)


In [ ]:
from pyngrok import ngrok
from google.colab import userdata

# Get the ngrok authtoken from Colab Secrets
#NGROK_AUTH_TOKEN = userdata.get('30DUrLJGkCoTTQQLANLW4q41i47_54edSjVdDSskBBvJD4kwS')

# Set the ngrok authtoken
ngrok.set_auth_token('30DUrLJGkCoTTQQLANLW4q41i47_54edSjVdDSskBBvJD4kwS')

# Connect to the Streamlit port (8501)
public_url = ngrok.connect(8501)
print("Streamlit URL:", public_url)

# Run the Streamlit app in the background
!streamlit run app.py &

Streamlit URL: NgrokTunnel: "https://c62bb7c3021b.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.135.184.100:8501

2025-07-22 06:05:35.687 Session with id 80616bec-9589-4617-9c17-39007ec52c1a is already connected! Connecting to a new session.


In [5]:
movie_name = input("Enter a movie name: ")
recommend(movie_name)


Enter a movie name: avatar


(['Aliens', 'Moonraker', 'Alien', 'Silent Running', 'Spaceballs'],
 ['https://image.tmdb.org/t/p/w500//r1x5JGpyqZU8PYhbs4UcrO1Xb6x.jpg',
  'https://image.tmdb.org/t/p/w500//6LrJdXNmu5uHOVALZxVYd44Lva0.jpg',
  'https://image.tmdb.org/t/p/w500//vfrQk5IPloGg1v9Rzbh2Eg3VGyM.jpg',
  'https://image.tmdb.org/t/p/w500//ahasZ5HILZHtlrl6cbXgpfk0y8H.jpg',
  'https://image.tmdb.org/t/p/w500//o624HTt93iIJIc1Sg5hNkDTnk5l.jpg'])